In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patheffects as path_effects


file_path = r"E:\Projects\Gamezone Orders Data\Data\Cleaned\gamezone_orders_data_cleaned.csv"
df = pd.read_csv(
    file_path,
    parse_dates=['purchase_ts_cleaned', 'ship_ts'],
    dtype={
        'purchase_year': 'Int64',
        'purchase_month': 'Int64',
        'time_to_ship': 'Int64',
        'revenue': 'float'
    },
    encoding='utf-8',
    keep_default_na=False,
    na_values=['']  # Only treat empty strings as NaN
)

In [3]:
# ✅ 1️⃣ TOP-SELLING PRODUCTS (by total revenue)
top_selling_products = (
    df.groupby('product_name_cleaned')['revenue']
    .sum()
    .reset_index()
    .rename(columns={'revenue': 'total_revenue'})
    .sort_values('total_revenue', ascending=False)
)
top_selling_products['revenue_share_%'] = (
    top_selling_products['total_revenue'] / top_selling_products['total_revenue'].sum() * 100
).round(2)

# ✅ 2️⃣ MOST PROFITABLE PRODUCTS (based on revenue per order)
profit_per_product = (
    df.groupby('product_name_cleaned')['revenue']
    .mean()
    .reset_index()
    .rename(columns={'revenue': 'avg_revenue_per_order'})
    .sort_values('avg_revenue_per_order', ascending=False)
)

# ✅ 3️⃣ AVERAGE PRICE PER PRODUCT
avg_price_per_product = (
    df.groupby('product_name_cleaned')['revenue']
    .mean()
    .reset_index()
    .rename(columns={'revenue': 'avg_price'})
)

# ✅ 4️⃣ PRODUCT REPEAT PURCHASE RATE (% of customers who ordered product >1 time)
repeat_customers = (
    df.groupby(['product_name_cleaned', 'user_id'])['order_id']
    .nunique()
    .reset_index()
)
repeat_rate = (
    repeat_customers.groupby('product_name_cleaned')
    .apply(lambda x: (x['order_id'] > 1).mean() * 100)
    .reset_index(name='repeat_purchase_rate_%')
)

# ✅ 5️⃣ MONTHLY REVENUE TREND PER PRODUCT
monthly_revenue_trend = (
    df.groupby(['product_name_cleaned', 'purchase_year', 'purchase_month'])['revenue']
    .sum()
    .reset_index()
    .rename(columns={'revenue': 'monthly_revenue'})
    .sort_values(['product_name_cleaned', 'purchase_year', 'purchase_month'])
)

# ✅ 6️⃣ PRODUCT PERFORMANCE BY CHANNEL & REGION
product_channel_region = (
    df.groupby(['product_name_cleaned', 'marketing_channel_cleaned', 'region'])['revenue']
    .sum()
    .reset_index()
    .rename(columns={'revenue': 'total_revenue'})
)

# ✅ 7️⃣ CREATE SUMMARY TABLE
product_summary = (
    top_selling_products[['product_name_cleaned', 'total_revenue', 'revenue_share_%']]
    .merge(profit_per_product, on='product_name_cleaned', how='left')
    .merge(avg_price_per_product, on='product_name_cleaned', how='left')
    .merge(repeat_rate, on='product_name_cleaned', how='left')

)

product_summary




C:\Users\shaif\AppData\Local\Temp\ipykernel_24460\556879204.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: (x['order_id'] > 1).mean() * 100)


,product_name_cleaned,total_revenue,revenue_share_%,avg_revenue_per_order,avg_price,repeat_purchase_rate_%
0,27in 4k gaming monitor,1968565.34,32.00,417.511207,417.511207,7.342818
1,nintendo switch,1658717.65,26.97,159.707072,159.707072,6.989469
2,sony playstation 5 bundle,1589243.85,25.84,1626.656960,1626.656960,8.444444
3,lenovo ideapad gaming 3,735506.56,11.96,1099.411898,1099.411898,8.265802
4,jbl quantum 100 gaming headset,96197.67,1.56,22.502379,22.502379,6.509317
5,acer nitro v gaming laptop,65661.18,1.07,754.726207,754.726207,11.538462
6,dell gaming mouse,36490.01,0.59,51.106457,51.106457,10.819165
7,razer pro gaming headset,884.23,0.01,126.318571,126.318571,0.000000


In [4]:
product_summary.to_csv('product_summary.csv', index=False)
monthly_revenue_trend.to_csv('product_monthly_trend.csv', index=False)
product_channel_region.to_csv('product_channel_region.csv', index=False)

In [5]:
product_channel_region

,product_name_cleaned,marketing_channel_cleaned,region,total_revenue
0,27in 4k gaming monitor,affiliate,APAC,2298.30
1,27in 4k gaming monitor,affiliate,EMEA,35201.80
2,27in 4k gaming monitor,affiliate,NA,50385.03
3,27in 4k gaming monitor,affiliate,unknown,338.78
4,27in 4k gaming monitor,direct,APAC,141601.89
...,...,...,...,...
114,sony playstation 5 bundle,social media,APAC,3191.62
115,sony playstation 5 bundle,social media,EMEA,3487.58
116,sony playstation 5 bundle,social media,NA,1800.00
117,sony playstation 5 bundle,unknown,APAC,3030.30
